# 03 SDK | 05 Chat Tools

## Tool Support

TODO TODO

GPT-4 Vision is a multi modal model capable of processing both text and images as input, allowing it to generate insights and information from diverse data sources. This model seamlessly integrates natural language understanding with visual information, enabling a more comprehensive and context-aware understanding of the input data. By handling both text and images, GPT-4 Vision opens up new possibilities for applications that require a holistic analysis of multi modal content.

### Further information

TODO

## Azure Environment

To execute the sample code Azure service specific information like endpoint, api key etc. is needed. ([Details and instructions can be found here](../01_DemoEnvironment/01_Environment.ipynb))


## Step 1: Create OpenAIClient

The OpenAIClient from Azure.AI.OpenAI is a .NET client library that acts as the centralized point for all .NET functionality that want to interact with a deployed Azure OpenAI Large Language Model. It provides methods to access the OpenAI REST APIs for various tasks such as text completion, text embedding, and chat completion, etc.. It also allows developers to specify the model, engine, and options for each request, such as temperature, frequency penalty, presence penalty, and stop sequences. 

The OpenAIClient can connect to any Azure OpenAI resource or to the non-Azure OpenAI inference endpoint, making it a versatile and powerful tool for .NET development with OpenAI.


In [21]:
#r "nuget: Azure.AI.OpenAI, 1.0.0-beta.12"
#r "nuget: DotNetEnv, 2.5.0"

using Azure; 
using Azure.AI.OpenAI;
using DotNetEnv;
using System.IO;
using System.Text.Json; 

//configuration file is created during environment creation
//if you skipped the deployment just remove the code and provide values from your deployment
static string _configurationFile = @"../01_DemoEnvironment/conf/application.env";
Env.Load(_configurationFile);

string oAiApiKey = Environment.GetEnvironmentVariable("SKIT_AOAI1106_APIKEY") ?? "SKIT_AOAI1106_APIKEY not found";
string oAiEndpoint = Environment.GetEnvironmentVariable("SKIT_AOAI1106_ENDPOINT") ?? "SKIT_AOAI1106_ENDPOINT not found";
string chatCompletionDeploymentName = Environment.GetEnvironmentVariable("SKIT_AOAI1106_DEPLOYMENTNAME") ?? "SKIT_AOAI1106_DEPLOYMENTNAME not found";
string assetsFolder = Path.Combine(Directory.GetCurrentDirectory(), "..", "..", "assets");

AzureKeyCredential azureKeyCredential = new AzureKeyCredential(oAiApiKey);
OpenAIClient openAIClient = new OpenAIClient(new Uri(oAiEndpoint), azureKeyCredential);


Console.WriteLine($"OpenAI Client created...");

Installed Packages Azure.AI.OpenAI, 1.0.0-beta.12 DotNetEnv, 2.5.0

OpenAI Client created...


Expected output:

```
Installed Packages
    Azure.AI.OpenAI, 1.0.0-beta.12
    DotNetEnv, 2.5.0

OpenAI Client created...
```

## Step 2: Compose ChatCompletionsOptions

Each chat would follow similar structure, where _System_, _Agent_ and _User_ messages are added in sequence. Parameters, such as _Temperature_ could be set per call.

TODO: Provide functions

In [22]:
//Define tool(s)
ChatCompletionsFunctionToolDefinition toolUserDetails = new ChatCompletionsFunctionToolDefinition() {
    Name = "SKit_Customer_GetCustomerDetails",
    Description = "Get customer details. Function returns all kind of information of registered customers this include first name and sure name",
    Parameters = BinaryData.FromObjectAsJson(
        new {
            Type = "object",
            Properties = new {
                Id = new {
                    Type = "string",
                    Description = "The customer Id (i.e. DK3-008AF)"
                }
            },
            Required = new[] { 
                "Id" 
            }
        },
        new JsonSerializerOptions() { PropertyNamingPolicy = JsonNamingPolicy.CamelCase }
    )   
}; 

ChatCompletionsFunctionToolDefinition toolOrderDetails = new ChatCompletionsFunctionToolDefinition() {
    Name = "SKit_Order_GetOrderDeliveryDate",
    Description = "Provides the expected delivery data of an order.",
    Parameters = BinaryData.FromObjectAsJson(
        new {
            Type = "object",
            Properties = new {
                OrderId = new {
                    Type = "string",
                    Description = "The ID of the order (i.e. 4711)"
                },
                CustomerId = new {
                    Type = "string",
                    Description = "The ID or name of the person who has ordered goods"
                }
            },
            Required = new[] { 
                "OrderId" 
            }
        },
        new JsonSerializerOptions() { PropertyNamingPolicy = JsonNamingPolicy.CamelCase }
    )   
}; 

Console.WriteLine("Tool: SDK_Order_GetOrderDeliveryDate() defined");
Console.WriteLine("Tool: SDK_Customer_GetCustomerDetails() defined");


Tool: SDK_Order_GetOrderDeliveryDate() defined
Tool: SDK_Customer_GetCustomerDetails() defined


Expected output: 
```
Tool: SDK_Order_GetOrderDeliveryDate() defined
Tool: SDK_Customer_GetCustomerDetails() defined
```

## Step 3: Define Prompt & ChatCompletionOptions

TODO TODO

In [24]:
//Define System Prompt
string systemMessage = @"You are an AI assistant that helps customers find information about their orders. 
    There's no need to ask for permission if you need additional tooling to fulfill the information request. 
    You answer with the first name and sure name of the customer.
    You always answer with: 'We expect your order to arrive at: {expected time of arrival}. This information was created for: {first name} {last name}'";

string userMessage = @"Hey, here's John.
    I'm calling on behalf of Contoso and I need information for regarding my order number 4711.
    The order was placed on March 1st where customer number is 984-AB489. Can you tell me when it will be delivered?
";

//Define chat completion options
var chatCompletionsOptions = new ChatCompletionsOptions()
{
    DeploymentName = chatCompletionDeploymentName,
    Messages = { 
        new ChatRequestSystemMessage(systemMessage),
        new ChatRequestUserMessage(userMessage) 
    },
    ToolChoice = ChatCompletionsToolChoice.Auto,
    Tools = { 
        toolUserDetails, 
        toolOrderDetails
    },
};


Console.WriteLine($"ChatCompletionsOptions created...");



ChatCompletionsOptions created...


Expected output:

```
ChatCompletionsOptions created...
```

## Step 4: Call ChatCompletion API

The final step is to call the `GetChatCompletionAsync()` method to get the response from the LLM.



In [25]:
//LLM Call
OpenAIClient openAIClient = new OpenAIClient(new Uri(oAiEndpoint), new AzureKeyCredential(oAiApiKey));
Response<ChatCompletions> response = await openAIClient.GetChatCompletionsAsync(chatCompletionsOptions);

//Tool Function Call
Console.WriteLine($"LLM call finish reason: {response.Value.Choices[0].FinishReason}");

List<ChatRequestToolMessage> chatRequestToolMessages = new List<ChatRequestToolMessage>();
ChatChoice chatChoice = response.Value.Choices[0]; 
if (chatChoice.FinishReason == CompletionsFinishReason.ToolCalls) {
    foreach (ChatCompletionsToolCall toolCall in chatChoice.Message.ToolCalls)
    {
        if (toolCall is ChatCompletionsFunctionToolCall functionToolCall)
        {
            if (functionToolCall != null)
            {
                Console.WriteLine($"Function call: {functionToolCall.Name} with arguments: {functionToolCall.Arguments}");
            }
        }
    }
}


LLM call finish reason: tool_calls
Function call: SKit_Customer_GetCustomerDetails with arguments: {"id": "984-AB489"}
Function call: SKit_Order_GetOrderDeliveryDate with arguments: {"orderId": "4711", "customerId": "984-AB489"}


Expected output:

```
LLM call finish reason: tool_calls
Function call: SKit_Customer_GetCustomerDetails with arguments: {"id": "984-AB489"}
Function call: SKit_Order_GetOrderDeliveryDate with arguments: {"orderId": "4711", "customerId": "984-AB489"}
```

## Step 5: Define Helper Class

In [26]:
using System.IO;
using System.Reflection;
using System.Text.Json;

#nullable enable
private string CallFunctionViaReflection(string name, string arguments)
{
    name = name.Replace("_", ".");
    string typeName = name.Substring(0, name.LastIndexOf("."));
    string methodName = name.Substring(name.LastIndexOf(".") + 1);

    string dllFilePath = Path.Combine(assetsFolder, "docs", "03_SDK", "FunctionCollection.dll");
    Assembly assembly = Assembly.LoadFrom(dllFilePath);
    Type? type = assembly.GetType(typeName);

    MethodInfo? method = type?.GetMethod(methodName);
    object? instance = null;
    if (type!=null) {
        instance = Activator.CreateInstance(type);
    }

    JsonElement jsonElement = JsonDocument.Parse(arguments).RootElement;
    int i = 0;
    if (method == null) { return ""; }
    object[] methodArguments = new object[method.GetParameters().Length];
    foreach (var parameter in method.GetParameters())
    {
        if (parameter.Name == null) { continue; }
        var parameterValue = jsonElement.GetProperty(parameter.Name).ToString();
        methodArguments[i++] = parameterValue;
    }

    if (method != null && instance != null) {
        object? result = method.Invoke(instance, methodArguments);
        return Convert.ToString(result) ?? "";
    }
    
    return ""; 
}

Console.WriteLine($"Helper to call function via reflection from dll defined...");


Helper to call function via reflection from dll defined...


## Steps 6: Call Local Function(s)

TODO:

In [27]:
List<ChatRequestToolMessage> chatRequestToolMessages = new List<ChatRequestToolMessage>();
ChatChoice chatChoice = response.Value.Choices[0]; 
if (chatChoice.FinishReason == CompletionsFinishReason.ToolCalls) {
    foreach (ChatCompletionsToolCall toolCall in chatChoice.Message.ToolCalls)
    {
        if (toolCall is ChatCompletionsFunctionToolCall functionToolCall)
        {
            if (functionToolCall != null)
            {
                string functionReturnValue = CallFunctionViaReflection(functionToolCall.Name, functionToolCall.Arguments);
                Console.WriteLine($"Function call: {functionToolCall.Name} with arguments: {functionToolCall.Arguments} returned {functionReturnValue}");
                chatRequestToolMessages.Add(
                    new ChatRequestToolMessage(
                        functionReturnValue, 
                        toolCall.Id
                    )
                );
            }
        }
    }
}

Function call: SKit_Customer_GetCustomerDetails with arguments: {"id": "984-AB489"} returned {"FirstName":"Jon","LastName":"Doe","Company":"Contoso","Title":"CTO"}
Function call: SKit_Order_GetOrderDeliveryDate with arguments: {"orderId": "4711", "customerId": "984-AB489"} returned 2/4/2024 6:27:58 PM


Expected output:

```
Function call: SKit_Customer_GetCustomerDetails with arguments: {"id": "984-AB489"} returned {"FirstName":"Jon","LastName":"Doe","Company":"Contoso","Title":"CTO"}
Function call: SKit_Order_GetOrderDeliveryDate with arguments: {"orderId": "4711", "customerId": "984-AB489"} returned 2/4/2024 6:23:22 PM
```

## Step 7: Finalize LLM Call

In [28]:
//Compose LLM Call
ChatChoice responseChoice = response.Value.Choices[0];
if (responseChoice.FinishReason == CompletionsFinishReason.ToolCalls)
{
    // Add the assistant message with tool calls to the conversation history
    ChatRequestAssistantMessage toolCallHistoryMessage = new(responseChoice.Message);
    chatCompletionsOptions.Messages.Add(toolCallHistoryMessage);

    // Add a new tool message for each tool call that is resolved
    foreach (var chatRequestToolMessage in chatRequestToolMessages) {
        chatCompletionsOptions.Messages.Add(chatRequestToolMessage);
    }

    response = await openAIClient.GetChatCompletionsAsync(chatCompletionsOptions);
    Console.WriteLine(response.Value.Choices[0].Message.Content); 
}


We expect your order to arrive at: 2/4/2024 6:27:58 PM. This information was created for: Jon Doe.


Expected output:

```
We expect your order to arrive at: 2/4/2024 6:23:22 PM. This information was created for: Jon Doe
```
